# Transfer data from instance to another

Sometimes, we want to load one instance just for reading data and use another to write data.

This is for instance, relevant if we only have read access to an instance.

You can do this with LaminDB via the `using` parameter.

## Setup

In [ ]:
!lamin init --storage ./test-transfer

In [ ]:
import lamindb as ln

## Query cellxgene-census

In [ ]:
dataset = ln.Dataset.filter(
    using="laminlabs/cellxgene-census", reference="c888b684-6c51-431f-972a-6c963044cef0"
).one()

In [ ]:
dataset

In [ ]:
dataset._state.db

In [ ]:
dataset._state.db = "default"

In [ ]:
dataset._state.db

In [ ]:
dataset.file_id = None
dataset.transform_id = None
dataset.run_id = None
dataset.created_by_id = ln.setup.settings.user.id

In [ ]:
dataset

In [ ]:
dataset.save()